In [ ]:
###############
### install ###
###############
'''

pip install pandas
pip install folium

'''

In [ ]:
#######################################
### creation and display of the map ###
#######################################

import folium
import pandas as pd
import random 
from folium import plugins
from folium import LayerControl
from folium import TileLayer
from folium.plugins import MeasureControl 
from folium.plugins import Fullscreen
from folium.plugins import HeatMap
from folium.plugins import Terminator

df = pd.read_csv('data_sentimental.csv')                        #Reading the csv file and storing it in a dataframe

try:                                                                                  #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    df.pop('Unnamed: 0')
except KeyError as err:
        pass

'''create a title'''
loc = 'Tweet about Bitcoin'                                                            #The title of the map
title_html = '''<h3 align="center" style="font-size:16px"><b>{}</h3>'''.format(loc)   

#qf = folium.Figure(width=1000, height=1000)                                            #Creating a figure with a width and height of 1000 pixels
m = folium.Map(location=[0, 0], zoom_start=2, control_scale=True)#.add_to(f)            #Creating a map 

m.get_root().html.add_child(folium.Element(title_html))                                #Adding the title to the map

m.add_child(MeasureControl())                                                          #Adding a measure control to the map

#nbTweet=int(input())                                                                  #Asking the user to input a number
nbTweet=25

displayTweet = random.sample(range(0, len(df)),nbTweet)                               #Creating a list of random numbers between 0 and the number of rows in the dataframe.

color = 'blue'                                                                        #The color of the marker
colorNeg= 'red'
colorPos='green'
'''lightgray', 'darkred', 'blue', 'pink', 'gray', 'red', 'orange', 'lightgreen', 'white', 'beige', 'darkblue', 'green', 'cadetblue', 'darkpurple', 'purple', 'lightblue', 'lightred', 'darkgreen', 'black'''

tooltipNeg = "View Negative tweet!"
tooltipPos = "View Positive tweet!"
tooltip = "View Neutral tweet!"

def popup_html(row):                                                                  #Creating a function that will be used to create the popups
    a = row
    name=df.user_name[a]                                                              #Creating a variable for column in the dataframe
    tweet=df.tweet[a]
    followers=df.user_followers[a]
    date=df.date[a] 
    hour=df.hour[a]                                                                   #The UTC datetime that the List was created on Twitter
    link=df.link[a]

    left_col_color = "#3498DB"                                                        #Setting the color of the left and right column of the table in the popup
    right_col_color = "#E1E8ED"
    
    #CreatinCreating a table with the tweet, link, followers, date and the hour
    #the html code for the popup
    html = """<!DOCTYPE html>
    <html>
    <h5 style="margin-bottom:0";margin-top: 0; width="10px">{}</h5>""".format(name) + """
        <table style="height: 100px; width: 250px;">
    <tbody>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Tweet<br><a href={} target="_blank" style="color:#FFC565;">Link</a></td>""".format(link) + """</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(tweet) + """
    </tr>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Followers</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(followers) + """
    </tr>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Date</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(date) + """
    </tr>
    <tr>
    <td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Hour (UTC)</span></td>
    <td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(hour) + """
    </tr>
    </tbody>
    </table>
    </html>
    """
    return html


i=0
while i < nbTweet :                                                                     #This is a while loop that will run until i is equal to nbTweet
    html = popup_html(displayTweet[i])                                                  #using the html created in the function popup_html
    size= min(max(200,len(str(df.tweet[displayTweet[i]]))*2.2),400)                     #definition of the size of the popup (min = 200, max=400) which takes into account the size of the tweet to display
    iframe = folium.IFrame(html,width=260,height=size)
    popup = folium.Popup(iframe,max_width=300)
    #print(df.sentiment[displayTweet[i]])
    if df.sentiment[displayTweet[i]]< -0.1:
        marker = folium.Marker([df.U_latitude[displayTweet[i]], df.U_longitude[displayTweet[i]]],popup=popup,icon=folium.Icon(color=colorNeg, icon='twitter', prefix='fa'),tooltip =tooltipNeg).add_to(m)        #creating and adding the marker with popup to the map
    
    elif df.sentiment[displayTweet[i]]> 0.1:
        marker = folium.Marker([df.U_latitude[displayTweet[i]], df.U_longitude[displayTweet[i]]],popup=popup,icon=folium.Icon(color=colorPos, icon='twitter', prefix='fa'),tooltip =tooltipPos).add_to(m)        #creating and adding the marker with popup to the map
    else :
        marker = folium.Marker([df.U_latitude[displayTweet[i]], df.U_longitude[displayTweet[i]]],popup=popup,icon=folium.Icon(color=color, icon='twitter', prefix='fa'),tooltip =tooltip).add_to(m)        #creating and adding the marker with popup to the map

    i+=1                                                                                 #add 1 to e to keep the index updated

'''Creating a heat map of the tweets'''
df.U_latitude=df.U_latitude.astype(float)                                                #Converting the latitude and longitude to float
df.U_longitude=df.U_longitude.astype(float)
heat_df=df[["U_latitude","U_longitude"]]                                                 #Creating a dataframe with only the columns U_latitude and U_longitude
heat_data=list(zip(df.U_latitude, df.U_longitude))                                       #Creating a list of tuples with the latitude and longitude
HeatMap(heat_data).add_to(folium.FeatureGroup(name='Heat Map',overlay=True, control=True, show=False).add_to(m))       #Creating and adding the heat map of the tweets to the map

'''Add dark and light mode '''
TileLayer(tiles='cartodbdark_matter',name="dark mode", opacity=0.8,control=True,show=True).add_to(m)     #Adding a dark mode to the map
TileLayer(tiles='cartodbpositron',name="light mode", opacity=0.8,control=True,show=True).add_to(m)
TileLayer(tiles='OpenStreetMap', opacity=0.7,control=True,show=True).add_to(m)

Terminator().add_to(folium.FeatureGroup(name='Day/Night',overlay=True, control=True, show=False).add_to(m))    #shows day and night which can be interesting with very recent tweets

'''Add a layer controller'''
#folium.LayerControl(collapsed=False).add_to(m)
LayerControl(position='topright', collapsed=False, autoZIndex=True, sortLayers=False).add_to(m)          #Adding a layer controller to the map

Fullscreen(position="topright",title="Expand me",title_cancel="Exit me",force_separate_button=True,).add_to(m) #Adding a fullscreen button to the map


'''Add a minimap'''
minimap = plugins.MiniMap()                                                                 #Adding a minimap to the map
m.add_child(minimap)


m.save('map-twiter.html')                                                                   #Saving the map as an html file

m                                                                                           #Display m, the map that we created

